In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://security.ubu

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-11 16:13:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-03-11 16:13:51 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [7]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [8]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
helpful_votes50_df = filtered_vine_df.withColumn('grater50', filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes']).filter('grater50>=0.5')
helpful_votes50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|0.8947368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             0.875|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|0.8846153846153846|
|R2M6WNM2OZZHKS|          1|  

In [9]:
# Step 3: Create a new DF: condition => vine=='Y'
vine_Y_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'Y')
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N| 0.918918918918919|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|0.8648648648648649|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|              0.88|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|          0.953125|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|               1.0|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|0.9655172413793104|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|0.9806201550387597|
|R13EGJI8C8YJ4Y|          5|  

In [10]:
# Step 4: Create a new DF: condition => vine=='N'
vine_N_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'N')
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|0.8947368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             0.875|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|0.8846153846153846|
|R2M6WNM2OZZHKS|          1|  

In [11]:
# Step 5.1: vine_reviews_YES: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews'
# Step 5.1.a: total number of reviews
vineY_reviews_TOTAL_df = vine_Y_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Reviews")
vineY_reviews_TOTAL_df.show()

+------------------+
|Total_Vine_Reviews|
+------------------+
|                61|
+------------------+



In [12]:
# Step 5.1.b: 5-star revievs 
vineY_reviews_5STAR_df = vine_Y_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_Vine_Reviews")
vineY_reviews_5STAR_df.show()

+--------------------+
|5-stars_Vine_Reviews|
+--------------------+
|                  20|
+--------------------+



In [13]:
vineY_reviews_percentage = vineY_reviews_5STAR_df.collect()[0]['5-stars_Vine_Reviews'] / vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews'] * 100
round(vineY_reviews_percentage,2)

32.79

In [14]:
# Step 5.2: vine_reviews_NO: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews '
# Step 5.2.a: total number of reviews
vineN_reviews_TOTAL_df = vine_N_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_non-Vine_Reviews")
vineN_reviews_TOTAL_df.show()

+----------------------+
|Total_non-Vine_Reviews|
+----------------------+
|                 28287|
+----------------------+



In [15]:
# Step 5.2.a: number of 5-stars reviews
vineN_reviews_5STAR_df = vine_N_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_non-Vine_Reviews")
vineN_reviews_5STAR_df.show()

+------------------------+
|5-stars_non-Vine_Reviews|
+------------------------+
|                   15689|
+------------------------+



In [16]:
vineN_reviews_percentage = vineN_reviews_5STAR_df.collect()[0]['5-stars_non-Vine_Reviews'] / vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews'] * 100
round(vineN_reviews_percentage,2)

55.46

In [17]:
# Create data frame for % results (5-stars rating % review)
percentage_df = spark.createDataFrame([
                                       (32.79, 55.46)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                  32.79|                      55.46|
+-----------------------+---------------------------+



In [22]:
# Positivity Bias - additional calculations
# Vine Reviews
all_stars_YES = vine_Y_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews Vine")
all_stars_YES.show()

+-----------+---------------------+
|star_rating|Count of Reviews Vine|
+-----------+---------------------+
|          1|                    7|
|          3|                   11|
|          5|                   20|
|          4|                   17|
|          2|                    6|
+-----------+---------------------+



In [23]:
# Vine Reviews (cont)
import pyspark.sql.functions as func
all_stars_YES_percentage = all_stars_YES.withColumn('% of total', func.round(all_stars_YES['Count of Reviews Vine']/ \
                                                (vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews']/100), 2)) \
                                                .orderBy(all_stars_YES['star_rating'].asc()).show()

+-----------+---------------------+----------+
|star_rating|Count of Reviews Vine|% of total|
+-----------+---------------------+----------+
|          1|                    7|     11.48|
|          2|                    6|      9.84|
|          3|                   11|     18.03|
|          4|                   17|     27.87|
|          5|                   20|     32.79|
+-----------+---------------------+----------+



In [25]:
# non-Vine Reviews
all_stars_NO = vine_N_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews non-Vine")
all_stars_NO.show()

+-----------+-------------------------+
|star_rating|Count of Reviews non-Vine|
+-----------+-------------------------+
|          1|                     6412|
|          3|                     1931|
|          5|                    15689|
|          4|                     2600|
|          2|                     1655|
+-----------+-------------------------+



In [29]:
# non-Vine Reviews (cont)
all_stars_NO_percentage = all_stars_NO.withColumn('% of total', func.round(all_stars_NO['Count of Reviews non-Vine']/ \
                                            (vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews']/100), 2))  \
                                            .orderBy(all_stars_NO['star_rating'].asc()).show()


+-----------+-------------------------+----------+
|star_rating|Count of Reviews non-Vine|% of total|
+-----------+-------------------------+----------+
|          1|                     6412|     22.67|
|          2|                     1655|      5.85|
|          3|                     1931|      6.83|
|          4|                     2600|      9.19|
|          5|                    15689|     55.46|
+-----------+-------------------------+----------+

